# Step 01 - Import Packages

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2 as cv
import os
import caer
import matplotlib.pyplot as plt
import random

from PIL import Image
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.preprocessing.image import img_to_array, array_to_img, ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Step 02 - Data Preprocessing

### Import data from Kaggle

In [ ]:
# ! mkdir ~/.kaggle
# ! cp /content/kaggle.json ~/.kaggle
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download -d vipoooool/new-plant-diseases-dataset

In [ ]:
# ! unzip /content/new-plant-diseases-dataset.zip

In [ ]:
# ! rm /content/new-plant-diseases-dataset.zip

### Data Evaluation and prerocessing

In [ ]:
plt.figure(figsize=(12, 12))
path = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Grape___Esca_(Black_Measles)'
for i in range(1, 12):
  plt.subplot(4, 4, i)
  plt.tight_layout()
  img = plt.imread(path+"/"+ random.choice(sorted(os.listdir(path))))
  plt.imshow(img)
plt.show()

In [ ]:
path_train = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'

In [ ]:

# x = []
# y = []
# for i in os.listdir(path):
#   new_path = os.path.join(path, i)
#   for j in os.listdir(new_path):
#     y.append(i)
#     img_path = os.path.join(new_path, j)
#     img = plt.imread(img_path)
#     img = cv.resize(img, (120, 120))
#     x.append(img)

In [ ]:
# y = np.array(y)
# y_encoder = LabelEncoder().fit(y)
# y = y_encoder.transform(y)
# x = np.array(x)

In [ ]:
image_generator = ImageDataGenerator(rescale=1/255.0)

In [ ]:
train = image_generator.flow_from_directory(path_train, 
                                            target_size=(256,256), 
                                            classes=os.listdir(path_train),
                                            batch_size=60
                                            )

In [ ]:
path_val = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

val = image_generator.flow_from_directory(path_val,
                                          target_size=(256, 256),
                                          classes=os.listdir(path_val),
                                          batch_size=60
                                          )

# Building the model

In [ ]:
model = Sequential()

In [ ]:
# adding layers
model.add(Conv2D(filters=128, kernel_size=(2, 2), 
                 activation='relu', 
                 input_shape=train.image_shape,
                 padding='same'))

model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


# adding layers
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', 
                 padding='same'))

model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))


# adding layers
model.add(Conv2D(filters=64, kernel_size=(4, 4), 
                 activation='relu', 
                 padding='same'))

model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# adding layers
model.add(Conv2D(filters=32, kernel_size=(2, 2), 
                 activation='relu', 
                 padding='same'))

model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

In [ ]:
model.add(Flatten())

# dense layers
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=38, activation='softmax'))

In [ ]:
# compile

model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# training the model
history = model.fit(train, epochs=15, validation_data=val)

In [ ]:
# evaluation of model

loss, score = model.evaluate(train)

In [ ]:
print("the Training data score", score)

In [ ]:
loss_t, score_t = model.evaluate(val)

In [ ]:
print("the Testing score", score_t)

In [ ]:
# creating model

img = plt.imread('./test/test/TomatoEarlyBlight6.JPG')
img = cv.resize(img, (256, 256))
resized  = img.reshape(1, 256, 256, 3)
print(os.listdir(path_train)[(np.argmax(model.predict(resized)))])
Image.fromarray(img)